In [28]:
!7za e models/my_model.7z  # models > /dev/null


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=es_CO.UTF-8,Utf16=on,HugeFiles=on,64 bits,8 CPUs Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz (806EA),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 75048254 bytes (72 MiB)

Extracting archive: models/my_model.7z
--
Path = models/my_model.7z
Type = 7z
Physical Size = 75048254
Headers Size = 130
Method = LZMA2:24
Solid = -
Blocks = 1

        
Would you like to replace the existing file:
  Path:     ./my_model.h5
  Size:     99801832 bytes (96 MiB)
  Modified: 2020-04-11 06:03:56
with the file from archive:
  Path:     my_model.h5
  Size:     99801832 bytes (96 MiB)
  Modified: 2020-04-11 06:03:56
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? ^C
(Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? 

#install tensorflow==2.1.0
#test to import tensor library
#install tensorflow-gpu==1.12.0 keras_preprocessin

In [1]:
import sys  
!{sys.executable} -m pip install tensorflow==2.0.0 h5py==2.9.0 ##necesaria para correr el  codigo

Defaulting to user installation because normal site-packages is not writeable


Test to import tensorflow library with GPU support

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Model, load_model

In [3]:
model = load_model('my_model.h5')

Step 2: Serve the prediction through a REST API

In [4]:
%%writefile serverDL.py
from flask import Flask, request, redirect, url_for, flash, jsonify
from tensorflow.keras.models import Model, load_model
import pickle
import json
import numpy as np

class NumpyEncoder(json.JSONEncoder):#function to coded the image
    def default(self,obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self,obj)


app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    json_load = json.loads(data)#cargar como json
    data = np.asarray(json_load["rimg"])
    data=np.array([data,data])
    #print(data.shape)
    #data=np.array(data)
    print ("LOG: predicting on input", data[0:1])
    preds_test = model.predict(data[0:1], verbose=1)
    json_dump=json.dumps({'pimg': preds_test,'rimg2': [2, (2, 3, 4), 1]}, cls=NumpyEncoder) 
    print(json_dump)
    #preds_test_t  = (preds_test > 0.5).astype(np.uint8)
    
    
    
    #prediction1 = np.array2string( preds_test_t)
    #prediction = np.array2string(model.predict(data[0:1]))
    #print ("LOG: returning predictions", prediction)
    return jsonify(json_dump)


if __name__ == '__main__':
    modelfile = 'my_model.h5'
    model = load_model('my_model.h5')
    print ("LOG: serving model", model.__class__.__name__)
    app.run(debug=True, host='127.0.0.1', port=7766)

Overwriting serverDL.py


In [5]:
import sys
e = sys.executable
e

'/usr/bin/python3'

In [6]:
pip = "/".join(e.split("/")[:-1])+"/pip"
!pip install flask

In [ ]:
!$e serverDL.py

2021-08-10 23:59:29.590733: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-08-10 23:59:29.612249: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 1999965000 Hz
2021-08-10 23:59:29.612681: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x541f000 executing computations on platform Host. Devices:
2021-08-10 23:59:29.612704: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
LOG: serving model Model
 * Serving Flask app "serverDL" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:7766/ (Press CTRL+C to quit)
 * Restarting with stat
2021-08-10 23:59:31.961516: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-08-10 23:59